# Load Dependencies and Data

In [1720]:
# Dependencies
import pandas as pd
import numpy as np
import datetime
from pathlib import Path

In [1721]:
# Store filepath in a variable
BigMac = "Resources/Original/BigmacPrice.csv"
MeatConsumption = "Resources/Original/meat_consumption.csv"
MinWage = "Resources/Original/Min_wage.csv"
AvgIncome = "Resources/Original/avg_income.csv"
Currency = "Resources/Original/currencies.csv"


In [1722]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
BigMac_df = pd.read_csv(BigMac)
MeatConsumption_df = pd.read_csv(MeatConsumption)
MinWage_df = pd.read_csv(MinWage)
AvgIncome_df = pd.read_csv(AvgIncome)
Currency_df = pd.read_csv(Currency)

# Preview data

In [1723]:
BigMac_df.head()


,date,currency_code,name,local_price,dollar_ex,dollar_price
0,4/1/00,ARS,Argentina,2.50,1,2.50
1,4/1/00,AUD,Australia,2.59,1,2.59
2,4/1/00,BRL,Brazil,2.95,1,2.95
3,4/1/00,GBP,Britain,1.90,1,1.90
4,4/1/00,CAD,Canada,2.85,1,2.85


In [1724]:
MeatConsumption_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,MEATCONSUMP,BEEF,KG_CAP,A,2000,26.515,NaN
1,AUS,MEATCONSUMP,BEEF,KG_CAP,A,2001,24.606,NaN
2,AUS,MEATCONSUMP,BEEF,KG_CAP,A,2002,25.263,NaN
3,AUS,MEATCONSUMP,BEEF,KG_CAP,A,2003,25.958,NaN
4,AUS,MEATCONSUMP,BEEF,KG_CAP,A,2004,28.719,NaN


In [1725]:
MinWage_df.head()

,COUNTRY,Country,SERIES,Series,PERIOD,Pay period,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,ESP,Spain,PPP,In 2021 constant prices at 2021 USD PPPs,H,Hourly,2001,2001,USD,US Dollar,0,Units,NaN,NaN,6.97,NaN,NaN
1,ESP,Spain,PPP,In 2021 constant prices at 2021 USD PPPs,H,Hourly,2002,2002,USD,US Dollar,0,Units,NaN,NaN,6.90,NaN,NaN
2,ESP,Spain,PPP,In 2021 constant prices at 2021 USD PPPs,H,Hourly,2003,2003,USD,US Dollar,0,Units,NaN,NaN,6.83,NaN,NaN
3,ESP,Spain,PPP,In 2021 constant prices at 2021 USD PPPs,H,Hourly,2004,2004,USD,US Dollar,0,Units,NaN,NaN,7.01,NaN,NaN
4,ESP,Spain,PPP,In 2021 constant prices at 2021 USD PPPs,H,Hourly,2005,2005,USD,US Dollar,0,Units,NaN,NaN,7.33,NaN,NaN


In [1726]:
AvgIncome_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,AVWAGE,TOT,USD,A,2000,45864.224684,NaN
1,AUS,AVWAGE,TOT,USD,A,2001,46325.645206,NaN
2,AUS,AVWAGE,TOT,USD,A,2002,46714.713801,NaN
3,AUS,AVWAGE,TOT,USD,A,2003,47349.662983,NaN
4,AUS,AVWAGE,TOT,USD,A,2004,48813.626970,NaN


In [1727]:
Currency_df.head()

,currency,currency_code
0,Afghani,AFN
1,Euro,EUR
2,Lek,ALL
3,Algerian Dinar,DZD
4,US Dollar,USD


# Create/cleanup tables for database

In [1728]:
#List of countries and codes from MinWage
country_codes = MinWage_df[['Country','COUNTRY']].drop_duplicates().sort_values(['Country']).reset_index(drop=True)
country_codes

,Country,COUNTRY
0,Australia,AUS
1,Belgium,BEL
2,Brazil,BRA
3,Canada,CAN
4,Chile,CHL
5,Colombia,COL
6,Costa Rica,CRI
7,Czech Republic,CZE
8,Estonia,EST
9,France,FRA


In [1729]:
#List of countries with currency codes from BigMac
currency_codes = BigMac_df[['name', 'currency_code']].drop_duplicates().sort_values(['name']).reset_index(drop=True)
currency_codes

,name,currency_code
0,Argentina,ARS
1,Australia,AUD
2,Austria,EUR
3,Azerbaijan,AZN
4,Bahrain,BHD
...,...,...
70,United States,USD
71,Uruguay,UYU
72,Venezuela,VEF
73,Venezuela,VES


In [1730]:
#Check to see if Russia, South Korea, and Turkey are included in currency codes list
print('Russia' in currency_codes['name'].unique())
print('South Korea'in currency_codes['name'].unique())
print('Turkey'in currency_codes['name'].unique())

True
True
True


In [1731]:
#Change spellings in country codes to match
country_codes = country_codes.replace(['Korea', 'Türkiye', 'Russian Federation'], ['South Korea', 'Turkey', 'Russia']) 

In [1732]:
#Create countries df with country, country code, and currency code
countries_df = pd.merge(country_codes, currency_codes, left_on='Country', right_on='name')
countries_df = countries_df.rename(columns={'COUNTRY': 'country_code', 'Country': 'country'}).drop(columns=['name']).set_index(['country']).sort_index()
countries_df

,country_code,currency_code
country,,
Australia,AUS,AUD
Belgium,BEL,EUR
Brazil,BRA,BRL
Canada,CAN,CAD
Chile,CHL,CLP
Colombia,COL,COP
Costa Rica,CRI,CRC
Czech Republic,CZE,CZK
Estonia,EST,EUR


In [1733]:
#create currency exchange table
#Change date to yyyy in BigMac_df
BigMac_df['date'] = pd.to_datetime(BigMac_df['date'])
BigMac_df['date'] = pd.DatetimeIndex(BigMac_df['date']).year
#merge countries_df and BigMac_df to make currency exchange table
CurrencyEx_df = pd.merge(BigMac_df, countries_df, on='currency_code').rename(columns={'date':'year'})
CurrencyEx_df = CurrencyEx_df[['currency_code', 'year', 'dollar_ex']]
#average dollar_ex for multiple entries in a year
CurrencyEx_df = CurrencyEx_df.groupby(['currency_code', 'year']).mean().reset_index().set_index('currency_code')
#preview cleaned df
CurrencyEx_df

,year,dollar_ex
currency_code,,
AUD,2000,1.0
AUD,2001,1.0
AUD,2002,1.0
AUD,2003,1.0
AUD,2004,1.0
...,...,...
USD,2018,1.0
USD,2019,1.0
USD,2020,1.0


In [1734]:
#Create table of unique currency codes
Currency_df = Currency_df[['currency_code', 'currency']]
Currency_df = Currency_df.drop_duplicates(['currency_code']).sort_values('currency_code')
Currency_df = pd.merge(Currency_df, countries_df, on='currency_code').drop(columns=['country_code']).drop_duplicates().set_index('currency_code')
Currency_df

,currency
currency_code,
AUD,Australian Dollar
BRL,Brazilian Real
CAD,Canadian Dollar
CLP,Chilean Peso
COP,Colombian Peso
CRC,Costa Rican Colon
CZK,Czech Koruna
EUR,Euro
HUF,Forint


In [1735]:
#BigMac_df cleanup
#rename 'name' to 'country' and 'date' to 'year'
BigMac_df = BigMac_df.rename(columns={"name": "country", "date": "year"})
BigMac_df = BigMac_df[['country', 'year', 'dollar_price']]

#only include countries in countries_df
BigMac_df = pd.merge(BigMac_df, countries_df, on='country').drop(columns=['country_code', 'currency_code']).drop_duplicates()
BigMac_df = BigMac_df.groupby(['country', 'year']).mean().reset_index()

#sort and preview cleaned up df
BigMac_df = BigMac_df.set_index(['country']).sort_index()
BigMac_df

,year,dollar_price
country,,
Australia,2000,2.590
Australia,2001,3.000
Australia,2002,3.000
Australia,2003,3.000
Australia,2004,3.250
...,...,...
United States,2018,4.615
United States,2019,4.710
United States,2020,4.820


In [1736]:
#MeatConsumption_df cleanup
#Rename 'location' to 'country code' and 'time' to 'year'
MeatConsumption_df = MeatConsumption_df.rename(columns={"LOCATION": "country_code", "TIME": "year", "Value": "kg_person"})
#Drop Indicator, Subject, Measure, Frequency, Flag Codes
MeatConsumption_df = MeatConsumption_df.drop(columns=['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'])
MeatConsumption_df = pd.merge(MeatConsumption_df, countries_df, on='country_code').drop(columns=['currency_code'])
#Preview cleaned table
MeatConsumption_df = MeatConsumption_df.set_index(['country_code']).sort_index().drop_duplicates()
MeatConsumption_df

,year,kg_person
country_code,,
AUS,2000,26.515
AUS,2021,19.187
AUS,2020,19.438
AUS,2019,19.698
AUS,2018,19.965
...,...,...
USA,2002,31.038
USA,2001,30.378
USA,2000,31.067


In [1737]:
#Pull in values for Country, TIME, and Value
MinWage_df = MinWage_df[['Country', 'TIME', 'Value']]
#Rename TIME to year and Value to hourly wage and Country to country
MinWage_df = MinWage_df.rename(columns={'TIME': 'year', 'Value': 'hourly_wage', 'Country': 'country'})
MinWage_df = pd.merge(MinWage_df, countries_df, on='country').set_index(['country']).sort_index().drop_duplicates().drop(columns=['country_code', 'currency_code'])
MinWage_df

,year,hourly_wage
country,,
Australia,2012,11.84
Australia,2005,11.51
Australia,2008,11.50
Australia,2009,11.65
Australia,2010,11.60
...,...,...
United States,2005,7.15
United States,2004,7.39
United States,2003,7.59


In [1738]:
#AvgIncome_df Cleanup
#Pull in location, time, and value columns
AvgIncome_df = AvgIncome_df[['LOCATION', 'TIME', 'Value']]
#rename columns for clarity and consistency
AvgIncome_df = AvgIncome_df.rename(columns={'LOCATION': 'country_code', 'TIME': 'year', 'Value': 'avg_income'})
#Round income column
AvgIncome_df = AvgIncome_df.round(2)
#Display preview of cleaned table
AvgIncome_df =pd.merge(AvgIncome_df, countries_df, on='country_code').set_index(['country_code']).sort_index().drop_duplicates().drop(columns='currency_code')
AvgIncome_df

,year,avg_income
country_code,,
AUS,2000,45864.22
AUS,2020,56454.64
AUS,2019,55170.16
AUS,2018,54585.74
AUS,2017,54446.19
...,...,...
USA,2018,67663.84
USA,2019,68842.31
USA,2020,72806.75


# Write cleaned data to new CSVs to import to SQL

In [1739]:
#Make a path for each df
CurrencyEx_filepath = Path('Resources/Cleaned/CurrencyEx.csv') 
Countries_filepath = Path('Resources/Cleaned/Countries.csv') 
BigMac_filepath = Path('Resources/Cleaned/BigMac.csv') 
MeatConsumption_filepath = Path('Resources/Cleaned/MeatConsumption.csv') 
MinWage_filepath = Path('Resources/Cleaned/MinWage.csv') 
AvgIncome_filepath = Path('Resources/Cleaned/AvgIncome.csv') 
Currency_filepath = Path('Resources/Cleaned/Currency.csv')

#write dfs to csvs via path
CurrencyEx_df.to_csv(CurrencyEx_filepath)
countries_df.to_csv(Countries_filepath)
BigMac_df.to_csv(BigMac_filepath)
MeatConsumption_df.to_csv(MeatConsumption_filepath)
MinWage_df.to_csv(MinWage_filepath)
AvgIncome_df.to_csv(AvgIncome_filepath)
Currency_df.to_csv(Currency_filepath)



